## Векторные представления слов и предложений (семинар 26.10.17)

**Дедлайны:** 9.00 утра 5 октября (очники), 9.00 утра 8 октября (заочники).

На этом семинаре мы проверим, способен ли простой подход на основе векторных представлений слов, находить вопросы-дубликаты на StackOverflow. Для этого мы подготовили для вас выборку дубликатов и выложили на страницу курса.


**План действий:**

* Скачаем предобученные вектора модели word2vec (с тем же успехом могли бы взять и вектора моделей GloVe, FastText, или любые другие).
* Усредним вектора слов в вопросе, чтобы получить представление всего вопроса.
* Отранжируем по вопросу-запросу набор из 100 случайных вопросов и 1 вопроса-дубликата. Будем использовать косинусную меру близости между векторами вопросов.
* На каком месте в выдаче окажется вопрос-дубликат? Оценим качество ранжирования.


В данном задании понадобятся:
- Предобученные вектора слов (GoogleNews-vectors-negative300)
- Данные о дубликатах вопросов с stackoverflow для анализа качества

In [2]:
import gensim
import numpy as np

### Загрузка предобученных векторов

Скачайте предобученные вектора https://code.google.com/archive/p/word2vec/ (GoogleNews-vectors-negative300) и создайте объект в gensim: https://radimrehurek.com/gensim/models/keyedvectors.html.

In [2]:
embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', 
                                                             binary=True, limit=500000)
# Your code here.
# 
#
#
###################

assert embeddings.most_similar(positive=['queen', 'man'], negative=['woman'], topn=1)[0][0] == 'king'

True

### Загрузка тестового датасета

В данном задании обучать ничего не нужно, поэтому достаточно загрузить только тестовый датасет. Это список пар очищенных вопросов-дубликатов со StackOverflow.

In [15]:
def read_question_duplicates(filepath):
    """Reads duplicate questions from file. filename. 
    
    Args:
        filepath: full path the the file with duplicate questions.
        
    Returns:
        questions (a list of strings): All questions from data.
        duplicate_idxs (a list of tuples of ints): Indices for duplicate questions.
    """
    
    questions_all = []
    duplicate_idxs = []
    current_idx = 0
    with open(filepath, 'r') as f:
        for row in f.readlines():
            questions = row.lower().strip().split("\t")
            assert len(questions) >= 2
            questions_all.extend(questions[:2])
            duplicate_idxs.append((current_idx, current_idx+1))
            current_idx += 2
    return questions_all, duplicate_idxs

In [16]:
questions, duplicate_idxs = read_question_duplicates('duplicate_questions.txt')

In [19]:
print(questions[:10])

['how to print a binary heap tree without recursion', 'how do you best convert a recursive function to an iterative one', 'phonestatelistener and service', 'how to start phonestatelistener programmatically', 'when hover on div1 depenting on if it is on div2 or not it should act differently', 'jquery show a div2 when mousenter over div1 is over', 'asynchronous sequence of events using promises', 'performing async method in a loop in node js and waiting for result', 'unreal engine 4 save rendered frame to memory', 'ue4 output game frames to file']


### Ранжирование и оценка качества

Если векторные представления для текстов (вопросов из stackoverflow) получаются хорошими, то косинусное расстояние между дубликатами должно получаться меньше, чем расстояние до случайного текста.
Поэтому для каждой пары дубликатов сгенерируем N случайных отрицательных текстов и посчитаем метрику DCG (https://en.wikipedia.org/wiki/Discounted_cumulative_gain). Будем считать, что релевантности случайных примеров равны 0, а релевантность истинного дубликата равна 1.

$$
DCG_k = \sum_{i=1}^k \frac{rel_i}{\log (1 + i)}   =\frac{1}{\log(1+rank_{dup})} \cdot [rank_{dup} < k],
$$

где $rank_{dup} \in [1, k]$ - позиция дубликата в отсортированном списке близости векторных представлений. Чем она больше, тем ближе дубликат по косинусному расстоянию к запросу и тем лучше наши векторные представления.

In [37]:
def cosine_distance(v1, v2):
    return 1 - np.sum(v1 * v2) / np.sqrt(np.sum(v1 * v1) * np.sum(v2 * v2))
    
def evaluate_dcg(object_vecs, duplicate_idxs, negative_idxs, k_values):
    """ 
    Ranks candidates by their embeddings and evaluates the ranking by DCG metric.
    
    Args:
        object_vecs (ndarray): Embeddings for all objects (questions).
        duplicate_idxs (list([ind1, ind2])): Duplicate indices (as defined by order in object_vecs).
        negative_idxs (list([ind_neg1, ... ind_negN])): Indices of negative objects for each duplicate pair.
        k_values (list): Several sizes of ranked lists for computing DCG@k.
    
    Returns:
    
        dcg_values (list): Computed values of DCG_at_k for each k (averaged over examples).
    """
    
    assert len(duplicate_idxs) == len(negative_idxs)
    
    # List (by a number of queries) of lists (by a number of different k) of dcg_at_k values. 
    dcg_values = []
    
    for (duplicate_ind1, duplicate_ind2), neg_indxs in zip(duplicate_idxs, negative_idxs):
        negative_size = len(neg_indxs)
        repeated_query = np.repeat(duplicate_ind1, negative_size + 1)
        candidates = np.hstack([duplicate_ind2, neg_indxs])
        distances = np.array([cosine_distance(object_vecs[candidates[i]], 
                                              object_vecs[repeated_query[i]]) for i in range(len(candidates))])
        rank = np.searchsorted(distances, distances[0]) + 1
        dcg_values.append([(rank <= k).astype(int) / np.log2(1 + rank) for k in k_values])
        
    # Average over different queries.
    dcg_values = np.mean(dcg_values, axis=0)
    return dcg_values

В качестве отрицательных примеров будем брать случайные вопросы:

In [25]:
np.random.seed(249)
negative_size = 100
random_negative_idxs = [np.random.choice([idx for idx in range(len(questions))
                                          if idx not in di], 
                                          replace=False, 
                                          size=negative_size) for di in duplicate_idxs]

### Построение векторных представлений вопросов

Самый простой способ получить векторное представление вопроса - усреднить вектора его слов. Релизуйте функцию ниже.

In [33]:
def question2vec_mean(questions, embeddings):
    result = np.zeros((len(questions), embeddings.vector_size))
    for i in range(len(questions)):
        words = questions[i].split(' ')
        vectors = []
        for word in words:
            if word in embeddings:
                vectors.append(embeddings[word])
        result[i] = np.mean(vectors, axis=0)
    return result 
    """ 
    Computes question embeddings by averaging word embeddings.
    
    Args:
      questions (list of strings): List of questions to be embedded.
      embeddings (gensim object): Pre-trained word embeddings.
      
    Returns:
      ndarray of shape [num_questions, embed_size] with question embeddings.
    """

In [32]:
lel = np.array([embeddings['andrew'], embeddings['loh']])
lel.shape
print(np.mean(lel, axis=0).shape)

(300,)


In [34]:
question_vecs = question2vec_mean(questions, embeddings)

/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [35]:
assert question_vecs.shape[1] == 300

Произведем оценку качества векторных представлений для вопросов.

In [38]:
dcg_values = evaluate_dcg(question_vecs, duplicate_idxs, random_negative_idxs, [1,5,10,100])

In [39]:
print(dcg_values[:10])

[ 0.65331529  0.66786518  0.6763846   0.72094283]


In [26]:
assert dcg_values[0] > 0.44

Теперь придумайте более продвинутый алгоритм, чтобы побить **порог 0.46 для DCG@1**. Например, можно подобрать схему взвешивания векторов слов, получше предобработать тестовые предложения, или придумать что-нибудь еще. 

In [40]:
def question2vec_advanced(questions, embeddings):
    result = np.zeros((len(questions), embeddings.vector_size))
    for i in range(len(questions)):
        words = questions[i].split(' ')
        vectors = []
        for word in words:
            if word in embeddings and len(word) > 2:
                vectors.append(embeddings[word])
        result[i] = np.mean(vectors, axis=0)
    return result 

In [29]:
question_vecs = question2vec_advanced(questions, embeddings)

In [ ]:
assert question_vecs.shape[1] == 300

In [ ]:
dcg_values = evaluate_dcg(question_vecs, duplicate_idxs, random_negative_idxs, [1,5,10,100])

In [212]:
assert dcg_values[0] > 0.46

На этом сдаваемая часть задания заканчивается, и начинается тьюториал по библиотеке Faiss для быстрого поиска ближайших соседей в нашем векторном пространстве слов.

## Tutorial: Faiss

**Следующий пункт не явялется обязательным, но рекомендуется для ознакомления.** Далее используется библиотека https://github.com/facebookresearch/faiss

Для оценки качества векторных представлений для предложений ранее, мы выбирали негативные примеры как рандомные предложения из корпуса. И так как рандомный пример скорее всего не близок к текущему (например из другого раздела на сайте stackoverflow), в среднем мы получаем хоршие значения метрики  DCG. Однако мы можем брать самые близкие примеры из корпусапо косинуснуму расстоянию, таким образом получая сильные отрицательные примеры.

Если мы для каждого вектора будем считать попарные расстояния с каждым вектором из корпуса, это займет очень много времени (N^2 для всех векторов). Библиотека faiss позволяет быстро и эффективно находить k-ближайших **dense** векторов по разным метрикам (L2, косинусное расстояние и т.д). Также данная библиотека поддерживает работу на GPU.

Быстро находить близкие dense-вектора полезно во многих задачах - поиск ближайших изображений, поиск кандидатов для рекомендаций и т.д.

In [32]:
import faiss

In [33]:
negative_size = 100

In [34]:
dim = 300
index = faiss.IndexFlatIP(dim)

Так как из коробки библиотека не нормирует вектора, нужно сначала их отнормировать, чтобы получать корректные косинусные расстояния.

In [ ]:
question_normed = question_vecs_mean_word / np.linalg.norm(question_vecs_mean_word, axis=1)[:, np.newaxis]
question_normed = question_normed.astype(np.float32)

Добавим вектора в индекс.

In [36]:
index.add(question_normed)

In [37]:
%%time
n_neighbors = negative_size + 2
_, neighbors_list = index.search(question_normed, n_neighbors)

CPU times: user 2min 29s, sys: 4min 12s, total: 6min 42s
Wall time: 25.6 s


Видим, что поиск 100 ближайших соседей для всех векторов в корпусе занял довольно мало времени.

Далее для каждой пары дубликатов найдем ближайшие отрицательные примеры как ближайшие к первому вектору.

In [38]:
strong_negative_idxs = [[idx for idx in neighbors_list[dup_ind1] if idx not in (dup_ind1, dup_ind2)][:negative_size] 
                        for dup_ind1, dup_ind2 in duplicate_idxs]

In [40]:
_ = evaluate_dcg(question_vecs, duplicate_idxs, strong_negative_idxs, [1,5,10,100])

18475it [00:13, 1388.99it/s]

DCG@1: 0.1411637347767253
DCG@5: 0.18647366877376192
DCG@10: 0.1979738058932608
DCG@100: 0.22392785361147613


Метрика DCG в данном случае получилась значительно меньше. Зато в реальных задачах часто лучше сравниваться не с рандомными примерами, а с отрицательными примерами на которых большой шанс ошибиться.


Далее посмотрим какие отрицательные примеры мы получаем в случае ближайших векторов.

In [44]:
print("question: '{}'\n".format(questions[0]))
print("duplicate question: '{}'\n".format(questions[1]))
for i in range(4):      
    print("negative sample question {}: '{}'".format(i, questions[strong_negative_idxs[0][i]]))

question: 'how to print a binary heap tree without recursion'

duplicate question: 'how do you best convert a recursive function to an iterative one'

negative sample question 0: 'how to find the height of a node in binary tree recursively'
negative sample question 1: 'how to delete all nodes of a binary search tree'
negative sample question 2: 'recursion and binary trees'
negative sample question 3: 'get all possible binary trees using prolog'


Как видим, в качестве отрицательных примеров мы взяли довольно похожие сообщения.